<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2/blob/main/model_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. 사용 데이터 확인

In [11]:
df = pd.read_csv("/content/drive/MyDrive/total_movie_rating.csv", index_col = 0)
df.head()

,user_id,movie_id,rating,timestamp,age,gender,occupation,movie_title,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,49,M,writer,Kolya (1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,305,242,5,886307828,23,M,programmer,Kolya (1996),0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,242,4,883268170,42,M,executive,Kolya (1996),0,0,...,0,0,0,0,0,0,0,0,0,0
3,234,242,4,891033261,60,M,retired,Kolya (1996),0,0,...,0,0,0,0,0,0,0,0,0,0
4,63,242,3,875747190,31,M,marketing,Kolya (1996),0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99991 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      99991 non-null  int64 
 1   movie_id     99991 non-null  int64 
 2   rating       99991 non-null  int64 
 3   timestamp    99991 non-null  int64 
 4   age          99991 non-null  int64 
 5   gender       99991 non-null  object
 6   occupation   99991 non-null  object
 7   movie_title  99991 non-null  object
 8   unknown      99991 non-null  int64 
 9   Action       99991 non-null  int64 
 10  Adventure    99991 non-null  int64 
 11  Animation    99991 non-null  int64 
 12  Children's   99991 non-null  int64 
 13  Comedy       99991 non-null  int64 
 14  Crime        99991 non-null  int64 
 15  Documentary  99991 non-null  int64 
 16  Drama        99991 non-null  int64 
 17  Fantasy      99991 non-null  int64 
 18  Film-Noir    99991 non-null  int64 
 19  Horror       99991 non-nu

# 3. CB모델링
1. 코사인 유사도 계산
2. 추천 로직 설계
3. 오프라인 성능 평가

## 1. 코사인 유사도 계산

In [13]:
# 영화ID별 영화명과 장르를 나타낸 데이터프레임 생성
movie_df = df.drop_duplicates(['movie_id']).drop(columns=['user_id', 'rating', 'timestamp', 'age', 'gender','occupation']).sort_values('movie_id').reset_index(drop=True)
movie_df

,movie_id,movie_title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1678,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1679,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# 코사인 유사도에 사용할 특성만 추출(CountVectorizer로 된 데이터 활용)
movie_cntvect = movie_df.drop(columns=['movie_id', 'movie_title'])
movie_cntvect

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1677,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1679,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# 코사인 유사도 계산
movie_similarity = pd.DataFrame(cosine_similarity(movie_cntvect, movie_cntvect), index=movie_df.movie_title, columns=movie_df.movie_title)
movie_similarity

movie_title,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
movie_title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000
GoldenEye (1995),0.000000,1.000000,0.577350,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.816497,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.00000,0.000000
Four Rooms (1995),0.000000,0.577350,1.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.00000,0.000000
Get Shorty (1995),0.333333,0.333333,0.000000,1.000000,0.333333,0.577350,0.408248,0.666667,0.577350,0.408248,...,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350,0.000000,0.408248,0.57735,0.577350
Copycat (1995),0.000000,0.333333,0.577350,0.333333,1.000000,0.577350,0.408248,0.333333,0.577350,0.408248,...,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350,0.408248,0.408248,0.00000,0.577350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mat' i syn (1997),0.000000,0.000000,0.000000,0.577350,0.577350,1.000000,0.707107,0.577350,1.000000,0.707107,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.707107,0.00000,1.000000
B. Monkey (1998),0.000000,0.408248,0.707107,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.00000,0.000000
Sliding Doors (1998),0.000000,0.000000,0.000000,0.408248,0.408248,0.707107,0.500000,0.408248,0.707107,0.500000,...,0.000000,0.707107,0.707107,0.707107,0.707107,0.707107,0.500000,1.000000,0.00000,0.707107


## 2. 추천 로직 설계
- 추천 로직
    1. 유저 ID별로 추천 진행
    2. timestamp를 기준으로 평가를 정렬
    3. train, test 데이터를 4:1 비율로 분류
    4. train 데이터의 가장 마지막 timestamp를 기준으로 영화 추천 진행

In [16]:
# 해당 유저id의 데이터를 불러오는 함수
def load_by_user (df, user_id) :
    return df[df['user_id'] == user_id].sort_values('timestamp')

In [17]:
# 테스트를 위한 샘플데이터 생성
sample_data = load_by_user(df, 276)
sample_data.head()

,user_id,movie_id,rating,timestamp,age,gender,occupation,movie_title,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
17360,276,258,5,874786337,21,M,student,Contact (1997),0,0,...,0,0,0,0,0,0,1,0,0,0
12925,276,300,4,874786338,21,M,student,Air Force One (1997),0,1,...,0,0,0,0,0,0,0,1,0,0
54522,276,328,4,874786366,21,M,student,Conspiracy Theory (1997),0,1,...,0,0,0,0,1,1,0,1,0,0
18902,276,294,4,874786366,21,M,student,Liar Liar (1997),0,0,...,0,0,0,0,0,0,0,0,0,0
10650,276,288,4,874786392,21,M,student,Scream (1996),0,0,...,0,0,1,0,0,0,0,1,0,0


In [18]:
# 학습용 데이터와 테스트용 데이터로 분리하는 함수(비율 = 4:1)
def split_data (df) : # train_test_split모듈 사용시 timestamp값 기준이 아닌 랜덤으로 분리되기 때문에 함수 별도로 작성
    data_len = len(df)
    train_len = round(data_len*0.8)
    train_data = df.iloc[:train_len, :]
    test_data = df.iloc[train_len:, :]
    return train_data, test_data

In [19]:
# 학습용 데이터와 테스트용 데이터 분리
train_data, test_data = split_data(sample_data)
train_data.tail()

,user_id,movie_id,rating,timestamp,age,gender,occupation,movie_title,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
53934,276,383,2,877934828,21,M,student,"Flintstones, The (1994)",0,0,...,0,0,0,0,0,0,0,0,0,0
78282,276,78,4,877934828,21,M,student,Free Willy (1993),0,0,...,0,0,0,0,0,0,0,0,0,0
16729,276,1083,3,877934891,21,M,student,Albino Alligator (1996),0,0,...,0,0,0,0,0,0,0,1,0,0
16545,276,203,4,877934910,21,M,student,Unforgiven (1992),0,0,...,0,0,0,0,0,0,0,0,0,1
91010,276,1471,2,877934947,21,M,student,Hideaway (1995),0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
# 학습용 데이터에서 가장 마지막 영화 제목 추출
test_movie = train_data.iloc[-1, :]['movie_title']
test_movie

'Hideaway (1995)'

In [21]:
# 가장 마지막 영화 제목을 기반으로 비슷한 유사도를 가진 영화 10개 추천
sim_movielist = movie_similarity[test_movie].sort_values(ascending=False).reset_index()
recommend_movielist = sim_movielist.drop(index = sim_movielist[sim_movielist['movie_title'] == test_movie].index).reset_index(drop = True)
recommend_movielist[:10]

,movie_title,Hideaway (1995)
0,Mute Witness (1994),1.0
1,Red Rock West (1992),1.0
2,"Assignment, The (1997)",1.0
3,Underworld (1997),1.0
4,Nikita (La Femme Nikita) (1990),1.0
5,"Fan, The (1996)",1.0
6,Turbulence (1997),1.0
7,Trial by Jury (1994),1.0
8,Malice (1993),1.0
9,"39 Steps, The (1935)",1.0


## 3. 오프라인 평가
- 평가 방식
    1. 유저ID별로 test데이터에 해당하는 영화의 개수만큼 추천
    2. 실제 영화와 추천한 영화를 바탕으로 precision@k 또는 recall@k 계산

In [22]:
# 테스트에 사용할 데이터 확인
test_data

,user_id,movie_id,rating,timestamp,age,gender,occupation,movie_title,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
87980,276,1210,2,877934988,21,M,student,Virtuosity (1995),0,0,...,0,0,0,0,0,0,1,1,0,0
896,276,238,5,877935060,21,M,student,Raising Arizona (1987),0,0,...,0,0,0,0,0,0,0,0,0,0
50863,276,1095,1,877935135,21,M,student,High School High (1996),0,0,...,0,0,0,0,0,0,0,0,0,0
30470,276,1000,2,877935262,21,M,student,Lightning Jack (1994),0,0,...,0,0,0,0,0,0,0,0,0,1
82320,276,734,1,877935262,21,M,student,Made in America (1993),0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94385,276,916,4,892436298,21,M,student,Lost in Space (1998),0,1,...,0,0,0,0,0,0,1,1,0,0
81976,276,316,4,892436314,21,M,student,As Good As It Gets (1997),0,0,...,0,0,0,0,0,0,0,0,0,0
97639,276,1483,3,892436354,21,M,student,"Man in the Iron Mask, The (1998)",0,1,...,0,0,0,0,0,1,0,0,0,0
94821,276,915,4,892436368,21,M,student,Primary Colors (1998),0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# 20개의 영화 추천
recom_list = recommend_movielist[:20]
recom_list

,movie_title,Hideaway (1995)
0,Mute Witness (1994),1.0
1,Red Rock West (1992),1.0
2,"Assignment, The (1997)",1.0
3,Underworld (1997),1.0
4,Nikita (La Femme Nikita) (1990),1.0
5,"Fan, The (1996)",1.0
6,Turbulence (1997),1.0
7,Trial by Jury (1994),1.0
8,Malice (1993),1.0
9,"39 Steps, The (1935)",1.0


In [24]:
# precision@k 계산 함수
def precision_k(test_data, recommend_movielist) : 
    movietitle_list = recommend_movielist['movie_title'].to_list()
    common_movie = test_data[test_data['movie_title'].isin(movietitle_list)]

    return len(common_movie) / len(recommend_movielist)

# recall@k 계산 함수 
def recall_k(test_data, recommend_movielist) : 
    movietitle_list = recommend_movielist['movie_title'].to_list()
    common_movie = test_data[test_data['movie_title'].isin(movietitle_list)]

    return len(common_movie) / len(test_data)

In [25]:
precision_k(test_data, recom_list)

0.05

In [26]:
recall_k(test_data, recom_list)

0.009615384615384616